# Reprocessing procedure for Las Campanas Observatory Swope data using BANZAI-Imaging
## Matt Daily

## Set up relevant runtime configuration

In [7]:
import logging
from glob import glob
import os
import re

from banzai.calibrations import make_master_calibrations
from banzai import settings
from banzai import dbs
from banzai.utils.stage_utils import run_pipeline_stages
from banzai.logs import set_log_level
import banzai.main

import pkg_resources
import requests
from astropy.io import fits

### We construct multiple runtime contexts, one for each night-obs. 

This is so that data taken on one night is reduced with calibrations from that same night. We don't want to mix them up.

In [9]:
set_log_level('DEBUG')
logger = logging.getLogger('banzai')

# Define the data directories on Apophis
# For each raw data dir, define an MEF-combined data dir
raw_data_dirs = ['/apophis/eng/rocks/Swope/ut220925/fits_raw/', '/apophis/eng/rocks/Swope/ut221002/fits_raw/']
mef_combined_data_dirs = [os.path.join(os.getcwd(), 'swope_data', 'mef_combined', '220925'),
                          os.path.join(os.getcwd(), 'swope_data', 'mef_combined', '221002')]

# Create the MEF data dirs if they don't exist
for path in mef_combined_data_dirs:
    if not os.path.exists(path):
        os.makedirs(path)
    
# Now construct our context objects. We want to use a separate DB for each data dir.
contexts = []

for combined_dir in mef_combined_data_dirs:
    os.environ['OPENTSDB_PYTHON_METRICS_TEST_MODE'] = 'True'

    settings.processed_path = os.path.join(os.getcwd(), 'test_data')
    settings.fpack=True
    settings.db_address = f"sqlite:///{os.path.join(combined_dir, 'test.db')}"
    settings.no_bpm = True
    settings.reduction_level = 91

    # set up the context object.
    context = banzai.main.parse_args(settings, parse_system_args=False)
    contexts.append(context)

### Take the data from each directory and combine it into MEFs
This branch of BANZAI expects that each amplifier have an extension in an MEF. BANZAI can then do the work of stitching them together.

In [3]:
for raw_dir, combined_dir in zip(raw_data_dirs, mef_combined_data_dirs):
    print(f"Combining files in {raw_dir}")
    
    prefix_set = set()
    for file_path in glob(os.path.join(raw_dir, '*.fits.fz')):
        # extract the unique prefixes from the raw data directory
        prefix_set.add(re.sub('c\d', '', file_path))

    # now create MEFs from the groupings!
    for prefix in prefix_set:
        filepaths = [prefix.replace('.fits.fz', f'c{i}.fits.fz') for i in [4,2,3,1]]

        image_hdus = fits.HDUList([fits.PrimaryHDU()])
        for path in filepaths:
            image_hdus.append(fits.open(path)['COMPRESSED_IMAGE'])

        if image_hdus[1].header['EXPTYPE'].upper() == 'FOCUS':
            continue

        # Now do some munging of the headers.
        # 1 is top right, rotate 90 counterclockwise and invert x (x -> -y, y-> -x)
        image_hdus[1].data = image_hdus[1].data.T[::-1, ::-1]
        image_hdus[1].header['DATASEC'] = '[129:2184,129:2176]'
        image_hdus[1].header['BIASSEC'] = '[1:128,1:2176]'
        image_hdus[1].header['DETSEC'] = '[2057:4112,2049:4096]'
        image_hdus[1].header.pop('TRIMSEC')

        # 2 is bottom left rotate 90 clockwise and invert x (y -> x x -> y)
        image_hdus[2].data = image_hdus[2].data.T
        image_hdus[2].header['DATASEC'] = '[1:2056,1:2048]'
        image_hdus[2].header['BIASSEC'] ='[2057:2084,1:2176]'
        image_hdus[2].header['DETSEC'] = '[1:2056,1:2048]'
        image_hdus[2].header.pop('TRIMSEC')

        # 3 bottom right rotate 90 counterclockwise (x-> -y, y - > x)
        image_hdus[3].data = image_hdus[3].data.T[:, ::-1]
        image_hdus[3].header['DATASEC'] = '[129:2184,1:2048]'
        image_hdus[3].header['BIASSEC'] ='[1:129,1:2176]'
        image_hdus[3].header['DETSEC'] = '[2057:4112,1:2048]'
        image_hdus[3].header.pop('TRIMSEC')

        # 4 is top left 270 deg  counterclockwise (x -> y, y->-x)
        image_hdus[4].data = image_hdus[4].data.T[::-1, :]
        image_hdus[4].header['DATASEC'] = '[1:2056,129:2176]'
        image_hdus[4].header['BIASSEC'] ='[2057:2184,1:2176]'
        image_hdus[4].header['DETSEC'] = '[1:2056, 2049:4096]'
        image_hdus[4].header.pop('TRIMSEC')

        # Copy all overlapping values into the main header
        for i in image_hdus[1].header:
            if i == 'COMMENT' or len(i) == 0:
                continue
            if image_hdus[2].header.get(i) == image_hdus[1].header.get(i):
                image_hdus[0].header[i] = image_hdus[1].header[i]

        image_hdus[0].header['TRIMSEC'] = '[1:4112,1:4096]'

        for i in range(1,5):
            image_hdus[i].header['RDNOISE'] = image_hdus[i].header['ENOISE']
            image_hdus[i].header['GAIN']  = image_hdus[i].header['EGAIN'] 

        # now create an HDUList and a unique filename to store the data in
        filename_suffix_obstype_mapping = {'Bias': 'b00',
                                           'Flat': 'f00',
                                           'Object': 'e00'}
        hdu_list = fits.HDUList(image_hdus)
        filename = f'{image_hdus[1].header["FILENAME"][:-2]}-{image_hdus[1].header["UT-DATE"].replace("-", "")}-{filename_suffix_obstype_mapping[image_hdus[1].header["EXPTYPE"]]}.fits.fz'
        hdu_list.writeto(os.path.join(combined_dir, filename), overwrite=True, output_verify='silentfix')

### Now construct the database instance for each night, and add the camera

In [12]:
for context, combined_data_dir in zip(contexts, mef_combined_data_dirs):
    if os.path.exists(os.path.join(combined_data_dir, 'test.db')):
        os.remove(os.path.join(combined_data_dir, 'test.db'))
    os.system(f'banzai_create_db --db-address={context.db_address}')
    os.system(f'banzai_add_site --site LCO --latitude -29.08300 --longitude -70.698005 --elevation 2280 --timezone -4 --db-address={context.db_address}')
    os.system(f'banzai_add_instrument --site LCO --camera Direct/4Kx4K-4 --name Direct/4Kx4K-4 --instrument-type swope-imager --db-address={context.db_address}')

In [13]:
instrument = dbs.get_instruments_at_site('LCO', context.db_address)[0]

### For each night, reduce and stack all bias files

In [14]:
for context, combined_data_dir in zip(contexts, mef_combined_data_dirs):
    bias_files = glob(os.path.join(combined_data_dir, '*b00*'))
    for bias_file in bias_files:
        run_pipeline_stages([{'path': bias_file}], context)

In [19]:
def mark_frames_as_good(filenames, context):
    for filename in glob(f'test_data/LCO/*/*/processed/{filenames}'):
        dbs.mark_frame(os.path.basename(filename), "good", db_address=context.db_address)

In [20]:
for context in contexts:
    mark_frames_as_good('*b91*', context)

In [21]:
for context in contexts:
    make_master_calibrations(instrument, 'Bias', '2022-09-20', '2022-10-03', context)

### For each night, reduce and stack all flat files

In [ ]:
for context, combined_data_dir in zip(contexts, mef_combined_data_dirs):
    flat_files = glob(os.path.join(combined_data_dir, '*f00*'))
    for flat_file in flat_files:
        run_pipeline_stages([{'path': flat_file}], context)

In [24]:
for context in contexts:
    mark_frames_as_good('*f91*', context)

In [25]:
for context in contexts:
    make_master_calibrations(instrument, 'Flat', '2022-09-20', '2022-10-03', context)

### Now reduce the science files

In [26]:
for context, combined_data_dir in zip(contexts, mef_combined_data_dirs):
    science_files = glob(os.path.join(combined_data_dir, '*e00*'))
    for science_file in science_files:
        run_pipeline_stages([{'path': science_file}], context)